In [45]:
import os #paths to file
import numpy as np # linear algebra
import pandas as pd # data processing
import warnings# warning filter
import scipy as sp #pivot egineering
import matplotlib.pyplot as plt 
import seaborn as sns

#ML model
from sklearn.metrics.pairwise import cosine_similarity

""
#default theme and settings
pd.options.display.max_columns
plt.style.use("ggplot")


In [26]:
#warning hadle
warnings.filterwarnings("always")
warnings.filterwarnings("ignore")

In [27]:
rating_path = "./Desktop/datasets/rating.csv"
anime_path = "./Desktop/datasets/anime.csv"

In [28]:
rating_df = pd.read_csv(rating_path)

In [29]:
rating_df.head()

,user_id,anime_id,rating
0,1,20,-1
1,1,24,-1
2,1,79,-1
3,1,226,-1
4,1,241,-1


In [30]:
anime_df = pd.read_csv(anime_path)
anime_df.head()

,anime_id,name,genre,type,episodes,rating,members
0,32281,Kimi no Na wa.,"Drama, Romance, School, Supernatural",Movie,1,9.37,200630
1,5114,Fullmetal Alchemist: Brotherhood,"Action, Adventure, Drama, Fantasy, Magic, Mili...",TV,64,9.26,793665
2,28977,Gintama°,"Action, Comedy, Historical, Parody, Samurai, S...",TV,51,9.25,114262
3,9253,Steins;Gate,"Sci-Fi, Thriller",TV,24,9.17,673572
4,9969,Gintama&#039;,"Action, Comedy, Historical, Parody, Samurai, S...",TV,51,9.16,151266


In [31]:
anime_df.shape

(12294, 7)

In [32]:
rating_df.shape

(7813737, 3)

In [33]:
anime_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 12294 entries, 0 to 12293
Data columns (total 7 columns):
 #   Column    Non-Null Count  Dtype  
---  ------    --------------  -----  
 0   anime_id  12294 non-null  int64  
 1   name      12294 non-null  object 
 2   genre     12232 non-null  object 
 3   type      12269 non-null  object 
 4   episodes  12294 non-null  object 
 5   rating    12064 non-null  float64
 6   members   12294 non-null  int64  
dtypes: float64(1), int64(2), object(4)
memory usage: 672.5+ KB


In [34]:
rating_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 7813737 entries, 0 to 7813736
Data columns (total 3 columns):
 #   Column    Dtype
---  ------    -----
 0   user_id   int64
 1   anime_id  int64
 2   rating    int64
dtypes: int64(3)
memory usage: 178.8 MB


In [35]:
print("Anime missing values (%): \n")
print(round(anime_df.isnull().sum().sort_values(ascending = False)/len(anime_df), 4) * 100)

Anime missing values (%): 

rating      1.87
genre       0.50
type        0.20
anime_id    0.00
name        0.00
episodes    0.00
members     0.00
dtype: float64


In [36]:
print("Rating missing values (%): \n")
print(round(rating_df.isnull().sum().sort_values(ascending = False)/len(rating_df), 4) * 100)

Rating missing values (%): 

user_id     0.0
anime_id    0.0
rating      0.0
dtype: float64


In [37]:
print(anime_df["genre"].mode())
print(anime_df["rating"].mode())
print(anime_df["type"].mode())

0    Hentai
Name: genre, dtype: object
0    6.0
Name: rating, dtype: float64
0    TV
Name: type, dtype: object


In [24]:
# uhm... so I guess we're just Hentai fans...?
# it looks like there're missing values for rating, genre, and type 

In [40]:
# deleting anime with 0 rating 
anime_df = anime_df.dropna(subset = ["rating"])

In [42]:
# filling in missing values for genre 
anime_df["genre"].fillna("Hentai", inplace = True)
# filling in missing values for type 
anime_df["type"].fillna("TV", inplace = True)

In [43]:
print("Anime missing values (%): \n")
print(round(anime_df.isnull().sum().sort_values(ascending = False)/len(anime_df), 4) * 100)

Anime missing values (%): 

anime_id    0.0
name        0.0
genre       0.0
type        0.0
episodes    0.0
rating      0.0
members     0.0
dtype: float64


In [47]:
# EDA 
anime_df.head()

,anime_id,name,genre,type,episodes,rating,members
0,32281,Kimi no Na wa.,"Drama, Romance, School, Supernatural",Movie,1,9.37,200630
1,5114,Fullmetal Alchemist: Brotherhood,"Action, Adventure, Drama, Fantasy, Magic, Mili...",TV,64,9.26,793665
2,28977,Gintama°,"Action, Comedy, Historical, Parody, Samurai, S...",TV,51,9.25,114262
3,9253,Steins;Gate,"Sci-Fi, Thriller",TV,24,9.17,673572
4,9969,Gintama&#039;,"Action, Comedy, Historical, Parody, Samurai, S...",TV,51,9.16,151266


In [49]:
anime_df["genre"][0]

'Drama, Romance, School, Supernatural'

In [52]:
anime_df["genre"].value_counts()

Hentai                                                      863
Comedy                                                      521
Music                                                       297
Kids                                                        197
Comedy, Slice of Life                                       174
                                                           ... 
Adventure, Drama, Historical, Mystery                         1
Action, Mecha, Music, Mystery, Romance, Sci-Fi                1
Historical, Romance, School, Shoujo                           1
Adventure, Comedy, Drama, Martial Arts, Samurai, Shounen      1
Hentai, Slice of Life                                         1
Name: genre, Length: 3229, dtype: int64

In [65]:
#huh.... ok. 
# looks like there are some animes with mixed types, so they werent classified properly 


In [67]:
rating_df["rating"] = rating_df["rating"].apply(lambda x: np.nan if x == -1 else x)

In [68]:
rating_df.head(20)

,user_id,anime_id,rating
0,1,20,NaN
1,1,24,NaN
2,1,79,NaN
3,1,226,NaN
4,1,241,NaN
5,1,355,NaN
6,1,356,NaN
7,1,442,NaN
8,1,487,NaN
9,1,846,NaN


In [69]:
anime_df["type"].value_counts()

TV         3671
OVA        3285
Movie      2297
Special    1671
ONA         652
Music       488
Name: type, dtype: int64

In [70]:
# lets only consider the TVs
anime_df = anime_df[anime_df["type"] == "TV"]

In [73]:
rated_anime = rating_df.merge(anime_df, left_on = "anime_id", right_on = "anime_id", suffixes= ['_user', ''])

In [74]:
rated_anime.head()

,user_id,anime_id,rating_user,name,genre,type,episodes,rating,members
0,1,20,NaN,Naruto,"Action, Comedy, Martial Arts, Shounen, Super P...",TV,220,7.81,683297
1,3,20,8.0,Naruto,"Action, Comedy, Martial Arts, Shounen, Super P...",TV,220,7.81,683297
2,5,20,6.0,Naruto,"Action, Comedy, Martial Arts, Shounen, Super P...",TV,220,7.81,683297
3,6,20,NaN,Naruto,"Action, Comedy, Martial Arts, Shounen, Super P...",TV,220,7.81,683297
4,10,20,NaN,Naruto,"Action, Comedy, Martial Arts, Shounen, Super P...",TV,220,7.81,683297


In [75]:
rated_anime = rated_anime[["user_id", "name", "rating"]]

In [76]:
rated_anime_7500 = rated_anime[rated_anime["user_id"] <= 7500]
# only consider the first 7500 users because we want to compute things 

In [77]:
rated_anime_7500.head()

,user_id,name,rating
0,1,Naruto,7.81
1,3,Naruto,7.81
2,5,Naruto,7.81
3,6,Naruto,7.81
4,10,Naruto,7.81


In [78]:
# how many distinct anime do we have ? what if the first 7500 are all naruto raters? 
rated_anime_7500["name"].value_counts()


Death Note                         4095
Shingeki no Kyojin                 3313
Sword Art Online                   3258
Code Geass: Hangyaku no Lelouch    2996
Elfen Lied                         2940
                                   ... 
Ultra B                               1
Pants Pankurou                        1
Jewelpet Sunshine                     1
Backkom                               1
Meimon! Daisan Yakyuubu               1
Name: name, Length: 2734, dtype: int64

In [81]:
# guess thats an okay distribution.. I think SAO > Death note ngl 
# now, maybe we could do One hot encoding ? 

In [85]:
# there are 2734 animes, we could use One Hot Encoding here 

In [89]:
len(rated_anime_7500["name"].unique())

2734

In [90]:
from sklearn.preprocessing import OneHotEncoder

cat_encoder = OneHotEncoder()
rated_anime_1hot = cat_encoder.fit_transform(rated_anime_7500[["name"]])

In [91]:
rated_anime_1hot.toarray()

array([[0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       ...,
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.]])

In [96]:
len(cat_encoder.categories_[0])

2734

In [98]:
rated_anime_1hot 

<549739x2734 sparse matrix of type '<class 'numpy.float64'>'
	with 549739 stored elements in Compressed Sparse Row format>

In [99]:
anime_similarity = cosine_similarity(rated_anime_1hot)

MemoryError: Unable to allocate 2.20 TiB for an array with shape (549739, 549739) and data type float64

In [100]:
# wow.... 

In [101]:
pivot = rated_anime_7500.pivot_table(index=['user_id'], columns=['name'], values='rating')
pivot.head()

name,.hack//Roots,.hack//Sign,.hack//Tasogare no Udewa Densetsu,009-1,07-Ghost,11eyes,12-sai.: Chicchana Mune no Tokimeki,3 Choume no Tama: Uchi no Tama Shirimasenka?,30-sai no Hoken Taiiku,91 Days,...,"Zone of the Enders: Dolores, I",Zukkoke Knight: Don De La Mancha,ef: A Tale of Melodies.,ef: A Tale of Memories.,gdgd Fairies,gdgd Fairies 2,iDOLM@STER Xenoglossia,s.CRY.ed,xxxHOLiC,xxxHOLiC Kei
user_id,,,,,,,,,,,,,,,,,,,,,
1,NaN,NaN,NaN,NaN,NaN,6.49,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
5,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,8.11,NaN


In [104]:
# step 1
pivot_n = pivot.apply(lambda x: (x-np.mean(x))/(np.max(x)-np.min(x)), axis=1)

# step 2
pivot_n.fillna(0, inplace=True)

# step 3
pivot_n = pivot_n.T

# step 4
pivot_n = pivot_n.loc[:, (pivot_n != 0).any(axis=0)]

# step 5
piv_sparse = sp.sparse.csr_matrix(pivot_n.values)


In [105]:
pivot_n.head()

user_id,1,2,3,4,5,6,7,8,10,11,...,7489,7490,7491,7492,7494,7495,7496,7497,7499,7500
name,,,,,,,,,,,,,,,,,,,,,
.hack//Roots,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
.hack//Sign,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,-0.179964,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
.hack//Tasogare no Udewa Densetsu,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
009-1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
07-Ghost,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [106]:
#model based on anime similarity
anime_similarity = cosine_similarity(piv_sparse)

#Df of anime similarities
ani_sim_df = pd.DataFrame(anime_similarity, index = pivot_n.index, columns = pivot_n.index)

In [107]:
ani_sim_df.head()

name,.hack//Roots,.hack//Sign,.hack//Tasogare no Udewa Densetsu,009-1,07-Ghost,11eyes,12-sai.: Chicchana Mune no Tokimeki,3 Choume no Tama: Uchi no Tama Shirimasenka?,30-sai no Hoken Taiiku,91 Days,...,"Zone of the Enders: Dolores, I",Zukkoke Knight: Don De La Mancha,ef: A Tale of Melodies.,ef: A Tale of Memories.,gdgd Fairies,gdgd Fairies 2,iDOLM@STER Xenoglossia,s.CRY.ed,xxxHOLiC,xxxHOLiC Kei
name,,,,,,,,,,,,,,,,,,,,,
.hack//Roots,1.000000,0.464082,0.445687,0.025392,0.069076,0.117115,0.005797,0.000000,0.056980,-0.023306,...,0.077054,0.0,-0.112560,-0.109070,0.000000,0.000000,0.071199,0.125517,-0.137733,-0.112823
.hack//Sign,0.464082,1.000000,0.443120,0.031732,0.093941,0.090260,0.004336,0.000000,0.069243,-0.024904,...,0.085169,0.0,-0.112569,-0.118462,0.000000,0.000000,0.036096,0.149564,-0.132065,-0.117723
.hack//Tasogare no Udewa Densetsu,0.445687,0.443120,1.000000,0.061698,0.057993,0.091261,0.007159,0.000000,0.049910,-0.023922,...,0.037200,0.0,-0.130604,-0.134231,0.000000,0.000000,0.065284,0.085838,-0.139700,-0.106539
009-1,0.025392,0.031732,0.061698,1.000000,0.006741,0.053089,0.013075,0.000000,0.003068,-0.031301,...,0.011599,0.0,-0.016247,-0.021515,0.000000,0.000000,0.000000,0.012534,-0.065892,-0.030031
07-Ghost,0.069076,0.093941,0.057993,0.006741,1.000000,0.175198,0.011276,0.000676,0.039599,-0.024257,...,0.011510,0.0,-0.086446,-0.084256,-0.000579,-0.000431,0.025849,0.049512,-0.097038,-0.108155


In [108]:
def anime_recommendation(ani_name):
    """
    This function will return the top 5 shows with the highest cosine similarity value and show match percent
    
    example:
    >>>Input: 
    
    anime_recommendation('Death Note')
    
    >>>Output: 
    
    Recommended because you watched Death Note:

                    #1: Code Geass: Hangyaku no Lelouch, 57.35% match
                    #2: Code Geass: Hangyaku no Lelouch R2, 54.81% match
                    #3: Fullmetal Alchemist, 51.07% match
                    #4: Shingeki no Kyojin, 48.68% match
                    #5: Fullmetal Alchemist: Brotherhood, 45.99% match 

               
    """
    
    number = 1
    print('Recommended because you watched {}:\n'.format(ani_name))
    for anime in ani_sim_df.sort_values(by = ani_name, ascending = False).index[1:6]:
        print(f'#{number}: {anime}, {round(ani_sim_df[anime][ani_name]*100,2)}% match')
        number +=1 

In [109]:
anime_recommendation('Sword Art Online')

Recommended because you watched Sword Art Online:

#1: Guilty Crown, 33.83% match
#2: Ao no Exorcist, 33.42% match
#3: Another, 30.93% match
#4: Akame ga Kill!, 30.91% match
#5: Naruto, 30.03% match
